In [1]:
import psycopg2
import pandas as pd

In [22]:
conn_hardy = psycopg2.connect("dbname='analytics' user='u_yidong' host='udacity-segment.c2zpsqalam7o.us-west-2.redshift.amazonaws.com' port='5439' password='Lyd580809?!'")

In [56]:
sql = """
WITH connect AS
  (SELECT interest_submit.anonymous_id,
          interest_submit.user_id,
          received_at::date,
          interest_submit.first_name,
          interest_submit.last_name,
          interest_submit.email,
          interest_submit.phone_number,
          ip_zipcode,
          nanodegree_interested,
          interested_location,
          interest_submit.start_dates,
          context_page_referrer,
          goal
   FROM frontend_endor.uconnect_interest_submitted AS interest_submit
   INNER JOIN analytics_tables.accounts AS accounts ON interest_submit.email = accounts.email
   UNION SELECT anonymous_id,
                user_id,
                received_at::date,
                user_first,
                user_last,
                email,
                NULL AS phone_number,
                zipcode,
                context_page_title AS nanodegree_interested,
                NULL AS interested_location,
                NULL AS start_dates,
                context_page_referrer,
                NULL AS goal
   FROM frontend_endor.nanodegree_notify_submitted AS notify_submit
   WHERE label = 'Nanodegree Notify - Uconnect Intensive' 
   UNION SELECT split_part(split_part(anonymous_id,': ',2),',',regexp_count(anonymous_id,',')+1) as anonymous_id,
                udacity_id,
                NULL AS received_at,
                firstname,
                lastname,
                email,
                phone,
                zip,
                nd_key,
                city AS location_interested,
                left(cohort_start_at,7) AS start_dates,
                split_part(split_part(anonymous_id,': ',2),',',regexp_count(anonymous_id,',')+1) AS context_page_referrer,
                goal
   FROM airflow.uconnect_lead_form_other_sources),

connect2 AS
  (SELECT *
      ,CASE WHEN LOWER(nanodegree_interested) LIKE 'nd%' THEN nanodegree_interested
        WHEN LOWER(nanodegree_interested) = 'ipnd' or LOWER(nanodegree_interested) LIKE '%intro%program%' THEN 'nd000'
        WHEN LOWER(nanodegree_interested) = 'fend' or LOWER(nanodegree_interested) LIKE '%front%end%' THEN 'nd001'
        WHEN LOWER(nanodegree_interested) = 'dand' or LOWER(nanodegree_interested) LIKE '%data%analyst%' THEN 'nd002'
        WHEN LOWER(nanodegree_interested) LIKE '%ios%developer%' THEN 'nd003'
        WHEN LOWER(nanodegree_interested) = 'fsnd' or LOWER(nanodegree_interested) LIKE '%full%stack%' THEN 'nd004'
        WHEN LOWER(nanodegree_interested) LIKE '%ios%development%' THEN 'nd006'
        WHEN LOWER(nanodegree_interested) LIKE '%entrepreneur%' THEN 'nd007'
        WHEN LOWER(nanodegree_interested) LIKE '%business%analy%' or LOWER(nanodegree_interested) LIKE '%analy%business%' THEN 'nd008'
        WHEN LOWER(nanodegree_interested) = 'mlnd' or LOWER(nanodegree_interested) LIKE '%machine%learning%' THEN 'nd009'
        WHEN LOWER(nanodegree_interested) = 'sdc' or LOWER(nanodegree_interested) LIKE '%self%driving%car%' THEN 'nd013'
        WHEN LOWER(nanodegree_interested) LIKE '%web%developer%' THEN 'nd016'
        WHEN LOWER(nanodegree_interested) LIKE '%virtual%reality%' THEN 'nd017'
        WHEN LOWER(nanodegree_interested) = 'dmnd' or LOWER(nanodegree_interested) LIKE '%digital%marketing%' THEN 'nd018'
        WHEN LOWER(nanodegree_interested) LIKE '%react%' THEN 'nd019'
        WHEN LOWER(nanodegree_interested) LIKE '%data%foundation%' THEN 'nd100'
        WHEN LOWER(nanodegree_interested) = 'dlnd' or LOWER(nanodegree_interested) LIKE '%deep%learn%' THEN 'nd101'
        WHEN LOWER(nanodegree_interested) LIKE '%robo%' THEN 'nd209'
        WHEN LOWER(nanodegree_interested) LIKE '%android%develop%' THEN 'nd801'
        WHEN LOWER(nanodegree_interested) LIKE '%android%basic%' THEN 'nd803'
        WHEN LOWER(nanodegree_interested) = 'aind' or LOWER(nanodegree_interested) LIKE '%artificial%intelligence%' THEN 'nd889'
       ELSE 'general interest'
       END AS nd
   FROM connect),

output AS
  (SELECT a.anonymous_id,
          a.context_page_referrer,
          a.user_id AS udacity_id,
          a.received_at AS createdate,
          a.first_name AS firstname,
          a.last_name AS lastname,
          a.email,
          a.phone_number AS phone,
          a.ip_zipcode AS zip,
          a.nanodegree_interested,
          a.interested_location AS location_interested,
          a.start_dates AS startdate_interested,
          a.goal AS what_is_your_goal,
          a.nd AS nd_key,
          CASE WHEN b.application_created_at >= coalesce(a.received_at,'1900-01-01') THEN b.application_status
          ELSE 'no applications' END AS application_status,
          CASE WHEN b.application_created_at >= coalesce(a.received_at,'1900-01-01') THEN b.application_created_at
          ELSE NULL END AS application_created_at,
          CASE WHEN b.application_created_at >= coalesce(a.received_at,'1900-01-01') THEN b.submitted_at
          ELSE NULL END AS application_submitted_at,
          CASE WHEN b.application_created_at >= coalesce(a.received_at,'1900-01-01') THEN b.accepted_at
          ELSE NULL END AS application_accepted_at,
          CASE WHEN b.application_created_at >= coalesce(a.received_at,'1900-01-01') THEN b.rejected_at
          ELSE NULL END AS application_rejected_at,
          CASE WHEN b.application_created_at >= coalesce(a.received_at,'1900-01-01') THEN b.first_charge_created_at
          ELSE NULL END AS application_paid_at,
          CASE WHEN b.application_created_at >= coalesce(a.received_at,'1900-01-01') THEN b.id
          ELSE NULL END AS application_id,
          CASE WHEN b.application_created_at >= coalesce(a.received_at,'1900-01-01') THEN b.cohort_id
          ELSE NULL END AS cohort_id,
          CASE WHEN b.application_created_at >= coalesce(a.received_at,'1900-01-01') THEN b.cohort_notify_at
          ELSE NULL END AS cohort_notify_at
   FROM connect2 a
   LEFT JOIN analytics_tables.applications b 
   ON a.user_id = b.applicant_id AND left(a.nd,5) = left(b.nd_key,5) and b.application_type = 'connect'
   WHERE NOT (a.email LIKE '%@udacity%')
  )

SELECT anonymous_id,
       case when context_page_referrer like '%http%' then 
            case when context_page_referrer like '%/' 
            then lower(substring(split_part(context_page_referrer,'://',2),0,len(context_page_referrer)))
            else lower(split_part(context_page_referrer,'://',2)) end
       else lower(context_page_referrer)
       end as context_page_referrer,
       udacity_id,
       createdate,
       firstname,
       lastname,
       email,
       phone,
       zip,
       nanodegree_interested,
       location_interested,
       startdate_interested,
       what_is_your_goal,
       nd_key,
       application_status,
       application_created_at,
       application_submitted_at,
       application_accepted_at,
       application_rejected_at,
       application_paid_at,
       application_id,
       cohort_id,
       cohort_notify_at
FROM output
union
SELECT anonymous_id,
       lower(anonymous_id) AS context_page_referrer,
       udacity_id,
       createdate,
       firstname,
       lastname,
       email,
       phone,
       zip,
       nanodegree_interested,
       location_interested,
       startdate_interested,
       what_is_your_goal,
       nd_key,
       application_status,
       application_created_at,
       application_submitted_at,
       application_accepted_at,
       application_rejected_at,
       application_paid_at,
       application_id,
       cohort_id,
       cohort_notify_at
FROM airflow.uconnect_applications
WHERE application_id NOT IN (SELECT DISTINCT application_id FROM output WHERE application_id IS NOT NULL); 
"""

In [58]:
df = pd.read_sql(sql,conn_hardy)

In [27]:
df

,anonymous_id,context_page_referrer,udacity_id,createdate,firstname,lastname,email,phone,zip,nanodegree_interested,...,nd_key,application_status,application_created_at,application_submitted_at,application_accepted_at,application_rejected_at,application_paid_at,application_id,cohort_id,cohort_notify_at
0,e1c738d3-35b0-49d7-8325-25f14fa0eefd,www.udacity.com/uconnect,1813198536,2018-01-22 00:00:00.000,Trinath,Patri,patri.trinath@gmail.com,None,72716,nd009,...,nd009,no applications,NaT,NaT,NaT,NaT,NaT,None,None,NaT
1,fa05036a-2f39-47d2-a537-83339cfcf3dc,www.google.com/,4173939541,2017-12-29 00:00:00.000,Said,Gaforov,sgaforov@gmail.com,3109776517,90501,Android,...,general interest,no applications,NaT,NaT,NaT,NaT,NaT,None,None,NaT
2,9dfb8cd4-1e7b-4554-9180-d8008a96327c,www.udacity.com/uconnect?%3Futm_source=faceboo...,None,2017-08-28 00:00:00.000,Jenny,Yu,jennycryu@gmail.com,None,95050,Udacity Connect Data Analyst,...,nd002,no applications,NaT,NaT,NaT,NaT,NaT,None,None,NaT
3,6081eddd-ee49-4aea-839b-13e0d7cf7797,www.udacity.com/uconnect,9744409234,2017-09-07 00:00:00.000,Megha,Maheshwari,meghaharlalka@gmail.com,None,94086,Udacity Connect Machine Learning,...,nd009,no applications,NaT,NaT,NaT,NaT,NaT,None,None,NaT
4,81809e5b-c205-4023-9718-aac72637ef66,payment.udacity.com/checkout/billing-info?prod...,5506222457,2017-09-19 00:00:00.000,Harshil,Shah,iamharshil@outlook.com,None,95825,Udacity Connect,...,general interest,no applications,NaT,NaT,NaT,NaT,NaT,None,None,NaT
5,d58e1040-12aa-46ee-a09e-3e0403a80b57,www.udacity.com/regulatory-information,None,2017-10-19 00:00:00.000,victor,trevino,vtrevino@terpmail.umd.edu,None,None,Udacity NY Connect Front-End Web Developer,...,nd001,no applications,NaT,NaT,NaT,NaT,NaT,None,None,NaT
6,app_created_aug,lead source: app_created_aug,9432961881,NaT,Ameya,Vilankar,ameya.vilankar@gmail.com,,94103,,...,general interest,no applications,NaT,NaT,NaT,NaT,NaT,None,None,NaT
7,created_submitted_app_Dec,lead source: created_submitted_app_Dec,9659930279,NaT,Geetha,Pandurangan,geethapandurangan@gmail.com,,94550,nd009,...,nd009,accept,2017-11-16 17:46:54.896,2017-11-16 18:00:31.418,2017-11-22 19:06:55.025,NaT,NaT,47cf81ea-1ea0-43d8-b6c6-ef618b874132,534,2017-11-29 00:16:00
8,app_created_aug,lead source: app_created_aug,1925918619,NaT,Bhoomi,Vyas,bhoomi.vyas@yahoo.com,,94588,,...,general interest,no applications,NaT,NaT,NaT,NaT,NaT,None,None,NaT
9,app_created_aug,lead source: app_created_aug,7424006341,NaT,Ken,Matsuda,ken_matsuda@hotmail.com,,,,...,general interest,no applications,NaT,NaT,NaT,NaT,NaT,None,None,NaT


In [ ]:
df.dtypes

In [55]:
df['context_page_referrer'] = [x.lower() if x.isnull()==False else x for x in df['context_page_referrer']]

AttributeError: 'str' object has no attribute 'isnull'

In [59]:
d = df.groupby(['context_page_referrer'])['anonymous_id'].count().rename('num_student').reset_index().sort_values(by='num_student',ascending=False)

In [60]:
app_created = df[df['application_created_at'].isnull()==False].groupby(['context_page_referrer'])['anonymous_id'].count().rename('num_app_created').reset_index().sort_values(by='num_app_created',ascending=False)

In [61]:
app_submitted = df[df['application_submitted_at'].isnull()==False].groupby(['context_page_referrer'])['anonymous_id'].count().rename('num_app_submitted').reset_index().sort_values(by='num_app_submitted',ascending=False)

In [62]:
o = pd.merge(pd.merge(d,app_created,on='context_page_referrer',how='left'),app_submitted,on='context_page_referrer',how='left').fillna(0)
o

,context_page_referrer,num_student,num_app_created,num_app_submitted
0,admissions-office,1702,1702.0,416.0
1,app_created_aug,345,68.0,14.0
2,app_created_oct,220,216.0,39.0
3,created_submitted_app_dec,200,181.0,54.0
4,info_session_oct_30,92,63.0,13.0
5,www.google.com/,84,4.0,2.0
6,created_app_aug,61,53.0,6.0
7,www.udacity.com/,49,2.0,1.0
8,www.udacity.com/uconnect,47,9.0,6.0
9,app_created_sept,38,23.0,15.0


In [63]:
o = o.assign(app_create_rate = o.apply(lambda row: row.num_app_created/row.num_student,axis=1))
o = o.assign(app_submit_rate = o.apply(lambda row: row.num_app_submitted/row.num_student,axis=1))

In [69]:
o.sort_values(by='app_create_rate',ascending=False)[['context_page_referrer','app_create_rate','num_app_created']]

,context_page_referrer,app_create_rate,num_app_created
0,admissions-office,1.000000,1702.0
13,app_created_sept_fend,1.000000,12.0
54,www.udacity.com/?utm_medium=email&utm_campaign...,1.000000,1.0
14,created_app_dand,1.000000,9.0
73,admissions.udacity.com/apply/nd009-connect,1.000000,1.0
2,app_created_oct,0.981818,216.0
3,created_submitted_app_dec,0.905000,181.0
6,created_app_aug,0.868852,53.0
12,created_app_oct,0.700000,14.0
4,info_session_oct_30,0.684783,63.0


In [68]:
o.sort_values(by='app_submit_rate',ascending=False)[['context_page_referrer','app_submit_rate','num_app_submitted']]

,context_page_referrer,app_submit_rate,num_app_submitted
13,app_created_sept_fend,0.916667,11.0
12,created_app_oct,0.600000,12.0
9,app_created_sept,0.394737,15.0
11,created_app_mlnd,0.291667,7.0
3,created_submitted_app_dec,0.270000,54.0
0,admissions-office,0.244418,416.0
21,www.udacity.com/course/machine-learning-engine...,0.200000,1.0
2,app_created_oct,0.177273,39.0
4,info_session_oct_30,0.141304,13.0
8,www.udacity.com/uconnect,0.127660,6.0
